# Run validation scripts in cmd
and keep history


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
import os
from pathlib import Path

In [3]:
pwd=!cd ../ && echo ${PWD}/
pwd = Path(pwd[0])
data_dir = (pwd / "input" / "as_mscoco")

os.environ['DATA_DIR'] = data_dir.as_posix()
os.environ['CUSTOM_DATASETS'] = (pwd / "datasets" / "dataset_catalog.py").as_posix()
base_output_path = Path("/home/storage_ntfs_1tb")
assert base_output_path.exists()

In [4]:
Detectron = (Path(".").absolute().parent / "Detectron.pytorch").as_posix()

In [5]:
def get_gt_bboxes_labels(image_id, annotations):
    selected_annotations = [a for a in gt_annotations if a['image_id'] == image_id]
    if len(selected_annotations) == 0:
        return []
    
    bboxes_labels = [(a['bbox'], str(a['category_id'])) for a in selected_annotations]
    return bboxes_labels

In [6]:
# !pip install --upgrade image-dataset-viz
from image_dataset_viz import render_datapoint, bbox_to_points, xywh_to_xyxy

In [7]:
def get_img_gt_target(img_path, image_id, annotations):
    img = Image.open(img_path / "{}.jpg".format(image_id))
    targets = [(bbox_to_points(xywh_to_xyxy(b)), l) for b, l in get_gt_bboxes_labels(image_id, gt_annotations)]
    return render_datapoint(img, targets, output_size=(430, 430))

## Validate models

Validation dataset contains: 41620 images

In [8]:
!find ${DATA_DIR}/val/ -name *.jpg | wc -l

41620


In [11]:
from datetime import datetime
now = datetime.now()

config_name = "finetune_faster_rcnn_X-101-64x4d-FPN_2x_gpu_1"
common_output_path = base_output_path / "output-OpenImagesObjectDetections" / config_name / "20180805_1459"
output_path = common_output_path / "val" / now.strftime("%Y%m%d_%H%M")
ckpt_weights_path = common_output_path / "ckpt" / "model_step24999.pth"
config = Path(".") / "configs" / "train" / "{}.yaml".format(config_name)
other_confs = "NUM_GPUS 1 "

!mkdir -p {output_path}
output_path = output_path.as_posix()
cmd = "{Detectron}/tools/test_net.py --cfg {config} --set {other_confs}".format(Detectron=Detectron, config=config, other_confs=other_confs) + \
    "--output_dir {output_path} --load_ckpt {ckpt_weights_path} ".format(output_path=output_path, ckpt_weights_path=ckpt_weights_path)

!echo python3 {cmd} > {output_path}/run.cmd
!python3 {cmd} > {output_path}/val.log

/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Traceback (most recent call last):
  File "/home/working_directory/ml/kaggle/OpenImagesObjectDetection/Detectron.pytorch/tools/test_net.py", line 100, in <module>
    check_expected_results=True)
  File "/home/working_directory/ml/kaggle/OpenImagesObjectDetection/Detectron.pytorch/lib/core/test_engine.py", line 128, in run_inference
    all_results = result_getter()
  File "/home/working_directory/ml/kaggle/OpenImagesObjectDetection/Detectron.pytorch/lib/core/test_engine.py", line 108, in result_getter
    multi_

In [20]:
detections_pkl = Path(output_path) / "detections.pkl"

In [14]:
import sys

sys.path.insert(0, (Path(".").resolve().parent / "Detectron.pytorch" / "lib").as_posix())

In [15]:
from core.config import cfg, cfg_from_file, assert_and_infer_cfg

cfg_from_file(config.as_posix())
assert_and_infer_cfg()

In [16]:
from datasets import task_evaluation

In [21]:
import pickle


with detections_pkl.open('rb') as fp:
    detections = pickle.load(fp, encoding='latin1')

In [34]:
from datasets.json_dataset import JsonDataset

dataset = JsonDataset('open_images_v4_val')

loading annotations into memory...
Done (t=0.75s)
creating index...
index created!


In [35]:
results = task_evaluation.evaluate_all(
    dataset, detections['all_boxes'], detections['all_segms'], detections['all_keyps'], output_path
)

Loading and preparing results...
DONE (t=7.87s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=341.40s).
Accumulating evaluation results...
DONE (t=62.92s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.020
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.035
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.020
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.023
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.039
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.058
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.059
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Test models

Test dataset contains: 125436 images

In [36]:
!find ${DATA_DIR}/test/ -name *.jpg | wc -l

125436


In [11]:
from datetime import datetime
now = datetime.now()

config_name = "finetune_faster_rcnn_X-101-64x4d-FPN_2x_gpu_1"
common_output_path = base_output_path / "output-OpenImagesObjectDetections" / config_name / "20180805_1459"
output_path = common_output_path / "val" / now.strftime("%Y%m%d_%H%M")
ckpt_weights_path = common_output_path / "ckpt" / "model_step24999.pth"
config = Path(".") / "configs" / "train" / "{}.yaml".format(config_name)
other_confs = "NUM_GPUS 1 "


!mkdir -p {output_path}
output_path = output_path.as_posix()
cmd = "{Detectron}/tools/test_net.py --cfg {config} --set {other_confs}".format(Detectron=Detectron, config=config, other_confs=other_confs) + \
    "--output_dir {output_path} --load_ckpt {ckpt_weights_path} ".format(output_path=output_path, ckpt_weights_path=ckpt_weights_path)

!echo python3 {cmd} > {output_path}/run.cmd
!python3 {cmd} > {output_path}/test.log

/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:1890: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Traceback (most recent call last):
  File "/home/working_directory/ml/kaggle/OpenImagesObjectDetection/Detectron.pytorch/tools/test_net.py", line 100, in <module>
    check_expected_results=True)
  File "/home/working_directory/ml/kaggle/OpenImagesObjectDetection/Detectron.pytorch/lib/core/test_engine.py", line 128, in run_inference
    all_results = result_getter()
  File "/home/working_directory/ml/kaggle/OpenImagesObjectDetection/Detectron.pytorch/lib/core/test_engine.py", line 108, in result_getter
    multi_